In [1]:
#### 1 - List of nominated movies to the following categories :
# Best Director (1927-2021)
# Best Actor (1927-2021)
# Best Actress (1927-2021)

import requests
from scrapy import Selector

list_response = [] ## LIST OF EVERY SELECTOR (scrapy) for every year of OSCARS

for i in range(1930,2022): # from 1930 (first Oscars to 2021 Oscars)

    url = "https://www.oscars.org/oscars/ceremonies/{}".format(i)
    response_oscars = requests.get(url)
    sel = Selector(text = response_oscars.text)   # Scrapy chose over BeautifulSoup for selector CSS
    list_response.append(sel)
    
print(list_response)


[<Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Selector xpath=None data='<html class="no-js" lang="en" dir="lt...'>, <Sele

In [3]:
### OOP   

class Oscar_Scaper:
    def __init__(self, time_period):
      self.links = {
          'pages':[],
          'something':[]
      }
      self.data = {
          'category':[],
          'film':[]
          'name':[],
          'gender':[],
          'result':[],
          'birthdate':[]
      }
    
    
      





SyntaxError: invalid syntax (<ipython-input-3-befceb31fb84>, line 12)

In [4]:
# For Best Director 
course_as = sel.css('#quicktabs-tabpage-honorees-0 > div > div.view-content > div.view-grouping > div.view-grouping-header > h2::text').extract()
print(course_as)

['Actor in a Leading Role', 'Actor in a Supporting Role', 'Actress in a Leading Role', 'Actress in a Supporting Role', 'Animated Feature Film', 'Art Direction', 'Cinematography', 'Costume Design', 'Directing', 'Documentary (Feature)', 'Documentary (Short Subject)', 'Film Editing', 'Foreign Language Film', 'Makeup', 'Music (Original Score)', 'Music (Original Song)', 'Best Picture', 'Short Film (Animated)', 'Short Film (Live Action)', 'Sound Editing', 'Sound Mixing', 'Visual Effects', 'Writing (Adapted Screenplay)', 'Writing (Original Screenplay)']


In [72]:
#### names of directors

list_directors = []
list_films = []
for i in range(len(course_as)):
    if course_as[i] == 'Directing':
        for number in range(int(number_of_film_nominated[1])+1):
            if number == 0:
             ### WINNER ####
                path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
                nominee_film = ''.join(sel.css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                print(nominee_film)
                list_films.append(nominee_film)
                path_name = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-title > span::text'.format(i+1)
                nominee_name = ''.join(sel.css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string
                nominee_name = nominee_name.replace("\n", "") # Cleaning up the string
                print(nominee_name)
                list_directors.append(nominee_name)
                
            else:
             ### NOMINEES ####
                path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-field-actor-name > h4::text".format(i+1, 3 +number)
                nominee_film = ''.join(sel.css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string4
                print(nominee_film)
                list_films.append(nominee_film)
                path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-title > span::text".format(i+1, 3 +number)
                nominee_name = ''.join(sel.css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string
                nominee_name = nominee_name.replace("\n", "") # Cleaning up the string
                print(nominee_name)
                list_directors.append(nominee_name)
print(list_directors)
print(list_films)

The Lord of the Rings: The Return of the King
Peter Jackson
City of God
Fernando Meirelles
Lost in Translation
Sofia Coppola
Master and Commander: The Far Side of the World
Peter Weir
Mystic River
Clint Eastwood
['Peter Jackson', 'Fernando Meirelles', 'Sofia Coppola', 'Peter Weir', 'Clint Eastwood']
['The Lord of the Rings: The Return of the King', 'City of God', 'Lost in Translation', 'Master and Commander: The Far Side of the World', 'Mystic River']


In [73]:
#### names of actors

list_actors = []
list_films = []
for i in range(len(course_as)):
    if course_as[i] == 'Actor in a Leading Role':
        for number in range(int(number_of_film_nominated[1])+1):
            if number == 0:
             ### WINNER ####
                path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
                nominee_film = ''.join(sel.css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                print(nominee_film)
                list_films.append(nominee_film)
                path_name = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-title > span::text'.format(i+1)
                nominee_name = ''.join(sel.css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string
                nominee_name = nominee_name.replace("\n", "") # Cleaning up the string
                print(nominee_name)
                list_actors.append(nominee_name)
                
            else:
             ### NOMINEES ####
                path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-field-actor-name > h4::text".format(i+1, 3 +number)
                nominee_film = ''.join(sel.css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string4
                print(nominee_film)
                list_films.append(nominee_film)
                path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-title > span::text".format(i+1, 3 +number)
                nominee_name = ''.join(sel.css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string
                nominee_name = nominee_name.replace("\n", "") # Cleaning up the string
                print(nominee_name)
                list_actors.append(nominee_name)
print(list_actors)
print(list_films)

Sean Penn
Mystic River
Johnny Depp
Pirates of the Caribbean: The Curse of the Black Pearl
Ben Kingsley
House of Sand and Fog
Jude Law
Cold Mountain
Bill Murray
Lost in Translation
['Mystic River', 'Pirates of the Caribbean: The Curse of the Black Pearl', 'House of Sand and Fog', 'Cold Mountain', 'Lost in Translation']
['Sean Penn', 'Johnny Depp', 'Ben Kingsley', 'Jude Law', 'Bill Murray']


In [74]:
#### names of actress

list_actress = []
list_films = []
for i in range(len(course_as)):
    if course_as[i] == 'Actress in a Leading Role':
        for number in range(int(number_of_film_nominated[1])+1):
            if number == 0:
             ### WINNER ####
                path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
                nominee_film = ''.join(sel.css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                print(nominee_film)
                list_films.append(nominee_film)
                path_name = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-title > span::text'.format(i+1)
                nominee_name = ''.join(sel.css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string
                nominee_name = nominee_name.replace("\n", "") # Cleaning up the string
                print(nominee_name)
                list_actors.append(nominee_name)
                
            else:
             ### NOMINEES ####
                path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-field-actor-name > h4::text".format(i+1, 3 +number)
                nominee_film = ''.join(sel.css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string4
                print(nominee_film)
                list_films.append(nominee_film)
                path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-title > span::text".format(i+1, 3 +number)
                nominee_name = ''.join(sel.css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string
                nominee_name = nominee_name.replace("\n", "") # Cleaning up the string
                print(nominee_name)
                list_actors.append(nominee_name)
print(list_actors)
print(list_films)

Charlize Theron
Monster
Keisha Castle-Hughes
Whale Rider
Diane Keaton
Something's Gotta Give
Samantha Morton
In America
Naomi Watts
21 Grams
['Mystic River', 'Pirates of the Caribbean: The Curse of the Black Pearl', 'House of Sand and Fog', 'Cold Mountain', 'Lost in Translation', 'Monster', 'Whale Rider', "Something's Gotta Give", 'In America', '21 Grams']
['Charlize Theron', 'Keisha Castle-Hughes', 'Diane Keaton', 'Samantha Morton', 'Naomi Watts']


Art Direction 1929
Art Direction 1930
Art Direction 1931
Art Direction 1932
Art Direction 1933
Art Direction 1934
Art Direction 1935
Art Direction 1936
Actress 1937
Actress 1938
Actress 1939
Actress 1940
Actress 1941
Actress 1942
Actress 1943
Actress 1944
Actress 1945
Actress 1946
Actress 1947
Actress 1948
Actress 1949
Actress 1950
Actress 1951
Actress 1952
Actress 1953
Actress 1954
Actress 1955
Actress 1956
Actress 1957
Actress 1958
Actress 1959
Actress 1960
Actress 1961
Actress 1962
Actress 1963
Actress 1964
Actress 1965
Actress 1966
Actress 1967
Actress 1968
Actress 1969
Actress 1970
Actress 1971
Actress 1972
Actress 1973
Actress 1974
Actress 1975
Actress 1976
Actress in a Leading Role 1977
Actress in a Leading Role 1978
Actress in a Leading Role 1979
Actress in a Leading Role 1980
Actress in a Leading Role 1981
Actress in a Leading Role 1982
Actress in a Leading Role 1983
Actress in a Leading Role 1984
Actress in a Leading Role 1985
Actress in a Leading Role 1986
Actress in a Leadi

In [129]:
###IMPORTANT ####
 # Actor until 1976
 # Actor in a Leading Role since 1977
 # Same for Actress
    
### Conserver les responses plutôt que demander h24 au serveur

### PROBLEME : COMMENT GERER LES 2 WINNERS, existe plusieurs fois # Done

### PROBLEM : for DIRECTION, names and titles are swapped
    



SyntaxError: EOL while scanning string literal (<ipython-input-129-d870590ff2b8>, line 13)

In [144]:
path_number = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child(1) > div.view-grouping-content> div:last-child"
number_of_film_nominated = ''.join(sel.css(path_number).extract()).split("views-row-")    
print(number_of_film_nominated)

['<div class="views-row ', '4 ', 'even ', 'last">\n      \n  <div class="views-field views-field-field-actor-name">        <h4 class="field-content">Adolphe Menjou</h4>  </div>  \n  <div class="views-field views-field-title">        <span class="field-content">The Front Page\n</span>  </div>  \n  <div class="views-field views-field-edit-node">        <span class="field-content"></span>  </div>  </div>']


In [152]:
# Crawler 
import time 
list_actress = []
list_films = []

for years in range(len(list_response)):
    
    categories = list_response[years].css('#quicktabs-tabpage-honorees-0 > div > div.view-content > div.view-grouping > div.view-grouping-header > h2::text').extract()    
    for i in range(len(categories)):
        if categories[i] in ('Actress', 'Actress in a Leading Role'):
            # Need the number of winners (might be ties) and nominated
            
            
            path_nominated = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:last-child".format(i+1)
            number_of_film_nominated = ''.join(list_response[years].css(path_nominated).extract()).split("views-row-")    
            
            
            path_winner = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2)".format(i+1)
            number_of_film_winner = ''.join(list_response[years].css(path_winner).extract()).split("views-row-first views-row-last") # If 1 winner then it splits the string into 1 list of 2 elements, if 2 winners, it does nothing
            
            
            
                 ### WINNER(S) ####
            if len(number_of_film_winner) == 2: # 1 Winner
                print(1930 + years,"1 winner")
                # NAME
                path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
                nominee_name = ''.join(list_response[years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string
                list_actress.append(nominee_name)

                # FIlM
                path_film = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-title > span::text'.format(i+1)
                nominee_film = ''.join(list_response[years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                nominee_film = nominee_film.replace("\n", "") # Cleaning up the string
                list_films.append(nominee_film)
                
                for number in range(int(number_of_film_nominated[1])):   
                ### NOMINEES ####
                    # NAME
                    path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-field-actor-name > h4::text".format(i+1, 4 +number)
                    nominee_name = ''.join(list_response[years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4
  
                    list_actress.append(nominee_name)
                    
                    # FILM
                    path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-title > span::text".format(i+1, 4 +number)
                    nominee_film = ''.join(list_response[years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                    nominee_film = nominee_film.replace("\n", "") # Cleaning up the string

                    list_films.append(nominee_film)     
                    
            if len(number_of_film_winner) == 1: # 2 Winners
                print(1930 + years,"2 winner")
                ### WINNERS ###       
                # NAME
                path_name1 = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2) > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
                winner_name1 = ''.join(list_response[years].css(path_name1).extract()) # ''.join() transforms the list produced by the selector into a string4
                list_actress.append(winner_name1)
                
                path_name2 = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(3) > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
                winner_name2 = ''.join(list_response[years].css(path_name2).extract()) # ''.join() transforms the list produced by the selector into a string4
                list_actress.append(winner_name2)

                
                # FILM
                path_film1 = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2) > div.views-field.views-field-title > span::text'.format(i+1)
                path_film2 = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(3) > div.views-field.views-field-title > span::text'.format(i+1)

                winner1_film = ''.join(list_response[years].css(path_film1).extract()) # ''.join() transforms the list produced by the selector into a string
                winner1_film = winner1_film.replace("\n", "") # Cleaning up the string

                list_films.append(winner1_film)

                winner2_film = ''.join(list_response[years].css(path_film2).extract()) # ''.join() transforms the list produced by the selector into a string
                winner2_film = winner2_film.replace("\n", "") # Cleaning up the string

                list_films.append(winner2_film)

                for number in range(int(number_of_film_nominated[1])):   
                    #### NOMINEES ####
                    # NAME
                    path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-field-actor-name > h4::text".format(i+1, 5 +number)
                    nominee_name = ''.join(list_response[years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4

                    list_actress.append(nominee_name)
                    
                    # FILM
                    path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-title > span::text".format(i+1, 5 +number)
                    nominee_film = ''.join(list_response[years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                    nominee_film = nominee_film.replace("\n", "") # Cleaning up the string

                    list_films.append(nominee_film)     

     
print(list_actress)
print(list_films)

1930 1 winner
1931 1 winner
1932 1 winner
1933 1 winner
1934 1 winner
1935 1 winner
1936 1 winner
1937 1 winner
1938 1 winner
1939 1 winner
1940 1 winner
1941 1 winner
1942 1 winner
1943 1 winner
1944 1 winner
1945 1 winner
1946 1 winner
1947 1 winner
1948 1 winner
1949 1 winner
1950 1 winner
1951 1 winner
1952 1 winner
1953 1 winner
1954 1 winner
1955 1 winner
1956 1 winner
1957 1 winner
1958 1 winner
1959 1 winner
1960 1 winner
1961 1 winner
1962 1 winner
1963 1 winner
1964 1 winner
1965 1 winner
1966 1 winner
1967 1 winner
1968 1 winner
1969 2 winner
1970 1 winner
1971 1 winner
1972 1 winner
1973 1 winner
1974 1 winner
1975 1 winner
1976 1 winner
1977 1 winner
1978 1 winner
1979 1 winner
1980 1 winner
1981 1 winner
1982 1 winner
1983 1 winner
1984 1 winner
1985 1 winner
1986 1 winner
1987 1 winner
1988 1 winner
1989 1 winner
1990 1 winner
1991 1 winner
1992 1 winner
1993 1 winner
1994 1 winner
1995 1 winner
1996 1 winner
1997 1 winner
1998 1 winner
1999 1 winner
2000 1 winner
2001 1

In [153]:
## ACTOR

list_actor = []
list_films = []

for years in range(len(list_response)):
    
    categories = list_response[years].css('#quicktabs-tabpage-honorees-0 > div > div.view-content > div.view-grouping > div.view-grouping-header > h2::text').extract()    
    for i in range(len(categories)):
        if categories[i] in ('Actor', 'Actor in a Leading Role'):
            # Need the number of winners (might be ties) and nominated
            
            
            path_nominated = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:last-child".format(i+1)
            number_of_film_nominated = ''.join(list_response[years].css(path_nominated).extract()).split("views-row-")    
            
            
            path_winner = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2)".format(i+1)
            number_of_film_winner = ''.join(list_response[years].css(path_winner).extract()).split("views-row-first views-row-last") # If 1 winner then it splits the string into 1 list of 2 elements, if 2 winners, it does nothing
            
            
            
                 ### WINNER(S) ####
            if len(number_of_film_winner) == 2: # 1 Winner
                print(1930 + years,"1 winner")
                # NAME
                path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
                nominee_name = ''.join(list_response[years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string
                list_actor.append(nominee_name)

                # FIlM
                path_film = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-title > span::text'.format(i+1)
                nominee_film = ''.join(list_response[years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                nominee_film = nominee_film.replace("\n", "") # Cleaning up the string
                list_films.append(nominee_film)
                
                for number in range(int(number_of_film_nominated[1])):   
                ### NOMINEES ####
                    # NAME
                    path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-field-actor-name > h4::text".format(i+1, 4 +number)
                    nominee_name = ''.join(list_response[years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4
  
                    list_actor.append(nominee_name)
                    
                    # FILM
                    path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-title > span::text".format(i+1, 4 +number)
                    nominee_film = ''.join(list_response[years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                    nominee_film = nominee_film.replace("\n", "") # Cleaning up the string

                    list_films.append(nominee_film)     
                    
            if len(number_of_film_winner) == 1: # 2 Winners
                print(1930 + years,"2 winner")
                ### WINNERS ###       
                # NAME
                path_name1 = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2) > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
                winner_name1 = ''.join(list_response[years].css(path_name1).extract()) # ''.join() transforms the list produced by the selector into a string4
                list_actor.append(winner_name1)
                
                path_name2 = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(3) > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
                winner_name2 = ''.join(list_response[years].css(path_name2).extract()) # ''.join() transforms the list produced by the selector into a string4
                list_actor.append(winner_name2)

                
                # FILM
                path_film1 = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2) > div.views-field.views-field-title > span::text'.format(i+1)
                path_film2 = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(3) > div.views-field.views-field-title > span::text'.format(i+1)

                winner1_film = ''.join(list_response[years].css(path_film1).extract()) # ''.join() transforms the list produced by the selector into a string
                winner1_film = winner1_film.replace("\n", "") # Cleaning up the string

                list_films.append(winner1_film)

                winner2_film = ''.join(list_response[years].css(path_film2).extract()) # ''.join() transforms the list produced by the selector into a string
                winner2_film = winner2_film.replace("\n", "") # Cleaning up the string

                list_films.append(winner2_film)

                for number in range(int(number_of_film_nominated[1])):   
                    #### NOMINEES ####
                    # NAME
                    path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-field-actor-name > h4::text".format(i+1, 5 +number)
                    nominee_name = ''.join(list_response[years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4

                    list_actor.append(nominee_name)
                    
                    # FILM
                    path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-title > span::text".format(i+1, 5 +number)
                    nominee_film = ''.join(list_response[years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                    nominee_film = nominee_film.replace("\n", "") # Cleaning up the string

                    list_films.append(nominee_film)     

     
print(list_actor)
print(list_films)
    

1930 1 winner
1931 1 winner
1932 1 winner
1933 2 winner
1934 1 winner
1935 1 winner
1936 1 winner
1937 1 winner
1938 1 winner
1939 1 winner
1940 1 winner
1941 1 winner
1942 1 winner
1943 1 winner
1944 1 winner
1945 1 winner
1946 1 winner
1947 1 winner
1948 1 winner
1949 1 winner
1950 1 winner
1951 1 winner
1952 1 winner
1953 1 winner
1954 1 winner
1955 1 winner
1956 1 winner
1957 1 winner
1958 1 winner
1959 1 winner
1960 1 winner
1961 1 winner
1962 1 winner
1963 1 winner
1964 1 winner
1965 1 winner
1966 1 winner
1967 1 winner
1968 1 winner
1969 1 winner
1970 1 winner
1971 1 winner
1972 1 winner
1973 1 winner
1974 1 winner
1975 1 winner
1976 1 winner
1977 1 winner
1978 1 winner
1979 1 winner
1980 1 winner
1981 1 winner
1982 1 winner
1983 1 winner
1984 1 winner
1985 1 winner
1986 1 winner
1987 1 winner
1988 1 winner
1989 1 winner
1990 1 winner
1991 1 winner
1992 1 winner
1993 1 winner
1994 1 winner
1995 1 winner
1996 1 winner
1997 1 winner
1998 1 winner
1999 1 winner
2000 1 winner
2001 1

In [155]:
## Directors

list_director = []
list_films = []

for years in range(len(list_response)):
    
    categories = list_response[years].css('#quicktabs-tabpage-honorees-0 > div > div.view-content > div.view-grouping > div.view-grouping-header > h2::text').extract()    
    for i in range(len(categories)):
        if categories[i] == 'Directing':
            # Need the number of winners (might be ties) and nominated
            
            
            path_nominated = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:last-child".format(i+1)
            number_of_film_nominated = ''.join(list_response[years].css(path_nominated).extract()).split("views-row-")    
            
            
            path_winner = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2)".format(i+1)
            number_of_film_winner = ''.join(list_response[years].css(path_winner).extract()).split("views-row-first views-row-last") # If 1 winner then it splits the string into 1 list of 2 elements, if 2 winners, it does nothing
            
            
            
                 ### WINNER(S) ####
            if len(number_of_film_winner) == 2: # 1 Winner
                print(1930 + years,"1 winner")
                # NAME
                path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
                nominee_name = ''.join(list_response[years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string
                list_director.append(nominee_name)

                # FIlM
                path_film = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-title > span::text'.format(i+1)
                nominee_film = ''.join(list_response[years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                nominee_film = nominee_film.replace("\n", "") # Cleaning up the string
                list_films.append(nominee_film)
                
                for number in range(int(number_of_film_nominated[1])):   
                ### NOMINEES ####
                    # NAME
                    path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-field-actor-name > h4::text".format(i+1, 4 +number)
                    nominee_name = ''.join(list_response[years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4
  
                    list_director.append(nominee_name)
                    
                    # FILM
                    path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-title > span::text".format(i+1, 4 +number)
                    nominee_film = ''.join(list_response[years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                    nominee_film = nominee_film.replace("\n", "") # Cleaning up the string

                    list_films.append(nominee_film)     
                    
            if len(number_of_film_winner) == 1: # 2 Winners
                print(1930 + years,"2 winner")
                ### WINNERS ###       
                # NAME
                path_name1 = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2) > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
                winner_name1 = ''.join(list_response[years].css(path_name1).extract()) # ''.join() transforms the list produced by the selector into a string4
                list_director.append(winner_name1)
                
                path_name2 = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(3) > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
                winner_name2 = ''.join(list_response[years].css(path_name2).extract()) # ''.join() transforms the list produced by the selector into a string4
                list_director.append(winner_name2)

                
                # FILM
                path_film1 = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2) > div.views-field.views-field-title > span::text'.format(i+1)
                path_film2 = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(3) > div.views-field.views-field-title > span::text'.format(i+1)

                winner1_film = ''.join(list_response[years].css(path_film1).extract()) # ''.join() transforms the list produced by the selector into a string
                winner1_film = winner1_film.replace("\n", "") # Cleaning up the string

                list_films.append(winner1_film)

                winner2_film = ''.join(list_response[years].css(path_film2).extract()) # ''.join() transforms the list produced by the selector into a string
                winner2_film = winner2_film.replace("\n", "") # Cleaning up the string

                list_films.append(winner2_film)

                for number in range(int(number_of_film_nominated[1])):   
                    #### NOMINEES ####
                    # NAME
                    path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-field-actor-name > h4::text".format(i+1, 5 +number)
                    nominee_name = ''.join(list_response[years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4

                    list_director.append(nominee_name)
                    
                    # FILM
                    path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-title > span::text".format(i+1, 5 +number)
                    nominee_film = ''.join(list_response[years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                    nominee_film = nominee_film.replace("\n", "") # Cleaning up the string

                    list_films.append(nominee_film)     

     
print(list_director)
print(list_films)
    

1930 1 winner
1931 1 winner
1932 1 winner
1933 1 winner
1934 1 winner
1935 1 winner
1936 1 winner
1937 1 winner
1938 1 winner
1939 1 winner
1940 1 winner
1941 1 winner
1942 1 winner
1943 1 winner
1944 1 winner
1945 1 winner
1946 1 winner
1947 1 winner
1948 1 winner
1949 1 winner
1950 1 winner
1951 1 winner
1952 1 winner
1953 1 winner
1954 1 winner
1955 1 winner
1956 1 winner
1957 1 winner
1958 1 winner
1959 1 winner
1960 1 winner
1961 1 winner
1962 1 winner
1963 1 winner
1964 1 winner
1965 1 winner
1966 1 winner
1967 1 winner
1968 1 winner
1969 1 winner
1970 1 winner
1971 1 winner
1972 1 winner
1973 1 winner
1974 1 winner
1975 1 winner
1976 1 winner
1977 1 winner
1978 1 winner
1979 1 winner
1980 1 winner
1981 1 winner
1982 1 winner
1983 1 winner
1984 1 winner
1985 1 winner
1986 1 winner
1987 1 winner
1988 1 winner
1989 1 winner
1990 1 winner
1991 1 winner
1992 1 winner
1993 1 winner
1994 1 winner
1995 1 winner
1996 1 winner
1997 1 winner
1998 1 winner
1999 1 winner
2000 1 winner
2001 1

In [188]:
 ### COMPLETE FOR OSCAR.ORG
    
list_director = []
list_actress = []
list_actor =  []
list_year = []
list_category = []
list_films = []

for years in range(len(list_response)):
    
    categories = list_response[years].css('#quicktabs-tabpage-honorees-0 > div > div.view-content > div.view-grouping > div.view-grouping-header > h2::text').extract()    
    for i in range(len(categories)):
        if categories[i] in ('Directing', 'Actor','Actor in a Leading Role', 'Actress', 'Actress in a Leading Role'):
            
            # Need the number of winners (might be ties) and nominated          
            path_nominated = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:last-child".format(i+1)
            number_of_film_nominated = ''.join(list_response[years].css(path_nominated).extract()).split("views-row-")    
            
            
            path_winner = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2)".format(i+1)
            number_of_film_winner = ''.join(list_response[years].css(path_winner).extract()).split("views-row-first views-row-last") # If 1 winner then it splits the string into 1 list of 2 elements, if 2 winners, it does nothing
            
            
            
                 ### WINNER(S) ####
            if len(number_of_film_winner) == 2: # 1 Winner
                # NAME
                path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
                nominee_name = ''.join(list_response[years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string
                list_year.append(1930 + years)
                list_category.append(categories[i])
                
                if categories[i] == "Directing":
                    list_director.append(nominee_name)
                if categories[i] in ('Actor','Actor in a Leading Role'):
                    list_actor.append(nominee_name)
                if categories[i] in ('Actress', 'Actress in a Leading Role'):
                    list_actress.append(nominee_name)

                # FIlM
                path_film = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-title > span::text'.format(i+1)
                nominee_film = ''.join(list_response[years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                nominee_film = nominee_film.replace("\n", "") # Cleaning up the string
                list_films.append(nominee_film)
                
                for number in range(int(number_of_film_nominated[1])):   
                ### NOMINEES ####
                    # NAME
                    path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-field-actor-name > h4::text".format(i+1, 4 +number)
                    nominee_name = ''.join(list_response[years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4
                    list_year.append(1930 + years)         
                    list_category.append(categories[i])
                    if categories[i] == "Directing": 
                        list_director.append(nominee_name)
                    if categories[i] in ('Actor','Actor in a Leading Role'):
                        list_actor.append(nominee_name)
                    if categories[i] in ('Actress', 'Actress in a Leading Role'):
                        list_actress.append(nominee_name)
                    
                    # FILM
                    path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-title > span::text".format(i+1, 4 +number)
                    nominee_film = ''.join(list_response[years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                    nominee_film = nominee_film.replace("\n", "") # Cleaning up the string

                    list_films.append(nominee_film)     
                    
            if len(number_of_film_winner) == 1: # 2 Winners
                ### WINNERS ###       
                # NAME
                path_name1 = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2) > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
                winner_name1 = ''.join(list_response[years].css(path_name1).extract()) # ''.join() transforms the list produced by the selector into a string4
                list_year.append(1930 + years)
                list_category.append(categories[i])
                
                if categories[i] == "Directing": 
                    list_director.append(winner_name1)
                if categories[i] in ('Actor','Actor in a Leading Role'):
                    list_actor.append(winner_name1)
                if categories[i] in ('Actress', 'Actress in a Leading Role'):
                    list_actress.append(winner_name1)
                
                path_name2 = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(3) > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
                winner_name2 = ''.join(list_response[years].css(path_name2).extract()) # ''.join() transforms the list produced by the selector into a string4
                list_year.append(1930 + years)
                list_category.append(categories[i])
                    
                if categories[i] == "Directing": 
                    list_director.append(winner_name2)
                if categories[i] in ('Actor','Actor in a Leading Role'):
                    list_actor.append(winner_name2)
                if categories[i] in ('Actress', 'Actress in a Leading Role'):
                    list_actress.append(winner_name2)

                
                # FILM
                path_film1 = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2) > div.views-field.views-field-title > span::text'.format(i+1)
                path_film2 = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(3) > div.views-field.views-field-title > span::text'.format(i+1)

                winner1_film = ''.join(list_response[years].css(path_film1).extract()) # ''.join() transforms the list produced by the selector into a string
                winner1_film = winner1_film.replace("\n", "") # Cleaning up the string
                
                list_films.append(winner1_film)

                winner2_film = ''.join(list_response[years].css(path_film2).extract()) # ''.join() transforms the list produced by the selector into a string
                winner2_film = winner2_film.replace("\n", "") # Cleaning up the string
                
                list_films.append(winner2_film)

                for number in range(int(number_of_film_nominated[1])):   
                    #### NOMINEES ####
                    # NAME
                    path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-field-actor-name > h4::text".format(i+1, 5 +number)
                    nominee_name = ''.join(list_response[years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4
                    list_year.append(1930 + years)
                    list_category.append(categories[i])
                    
                    if categories[i] == "Directing": 
                        list_director.append(nominee_name)
                    if categories[i] in ('Actor','Actor in a Leading Role'):
                        list_actor.append(nominee_name)
                    if categories[i] in ('Actress', 'Actress in a Leading Role'):
                        list_actress.append(nominee_name)
                    
                    # FILM
                    path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-title > span::text".format(i+1, 5 +number)
                    nominee_film = ''.join(list_response[years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                    nominee_film = nominee_film.replace("\n", "") # Cleaning up the string

                    list_films.append(nominee_film)     

### DOESN'T WORK WELL BECAUSE DIRECTORS AND TITLE ARE REVERSED ON WEBSITE


In [169]:
print(list_films)

['In Old Arizona', 'Thunderbolt', 'Alibi', 'The Valiant', 'The Patriot', 'Coquette', 'Madame X', 'The Barker', 'The Letter', 'The Divine Lady', 'The Broadway Melody', 'Frank Lloyd', 'Lionel Barrymore', 'Harry Beaumont', 'Irving Cummings', 'Frank Lloyd', 'Frank Lloyd', 'Ernst Lubitsch', 'Disraeli', 'The Green Goddess', 'The Big House', 'The Big Pond', 'The Love Parade', 'Bulldog Drummond', 'Condemned', 'The Rogue Song', 'The Divorcee', "The Devil's Holiday", 'Sarah and Son', 'Anna Christie', 'Romance', 'Their Own Desire', 'The Trespasser', 'Lewis Milestone', 'Clarence Brown', 'Clarence Brown', 'Robert Leonard', 'Ernst Lubitsch', 'King Vidor', 'A Free Soul', 'Skippy', 'Cimarron', 'The Royal Family of Broadway', 'The Front Page', 'Min and Bill', 'Morocco', 'Cimarron', 'Holiday', 'A Free Soul', 'Norman Taurog', 'Wesley Ruggles', 'Clarence Brown', 'Lewis Milestone', 'Josef Von Sternberg', 'The Champ', 'Dr. Jekyll and Mr. Hyde', 'The Guardsman', 'The Sin of Madelon Claudet', 'Emma', 'The Gua

In [189]:
### COMPLETE FOR OSCAR.ORG ALTERNATIVE WITH ONLY LIST_NAMES
    
list_names = []
list_year = []
list_category = []
list_films = []
list_results = []

for years in range(len(list_response)):
    
    categories = list_response[years].css('#quicktabs-tabpage-honorees-0 > div > div.view-content > div.view-grouping > div.view-grouping-header > h2::text').extract()    
    for i in range(len(categories)):
        if categories[i] in ('Directing', 'Actor','Actor in a Leading Role', 'Actress', 'Actress in a Leading Role'):
            
            # Need the number of winners (might be ties) and nominated          
            path_nominated = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:last-child".format(i+1)
            number_of_film_nominated = ''.join(list_response[years].css(path_nominated).extract()).split("views-row-")    
            
            
            path_winner = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2)".format(i+1)
            number_of_film_winner = ''.join(list_response[years].css(path_winner).extract()).split("views-row-first views-row-last") # If 1 winner then it splits the string into 1 list of 2 elements, if 2 winners, it does nothing
            
            
            
                 ### WINNER(S) ####
            if len(number_of_film_winner) == 2: # 1 Winner
                # NAME
                path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
                nominee_name = ''.join(list_response[years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string
                list_year.append(1930 + years)
                list_category.append(categories[i])
                list_results.append("Winner")
                if categories[i] == "Directing": # For Directing, titles and names are swapped on the website
                    list_films.append(nominee_name)
                else:
                    list_names.append(nominee_name)

                # FIlM
                path_film = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div.views-row.views-row-1.views-row-odd.views-row-first.views-row-last > div.views-field.views-field-title > span::text'.format(i+1)
                nominee_film = ''.join(list_response[years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                nominee_film = nominee_film.replace("\n", "") # Cleaning up the string
                if categories[i] == "Directing":# For Directing, titles and names are swapped on the website
                    list_names.append(nominee_film)
                else:
                    list_films.append(nominee_film)
                
                for number in range(int(number_of_film_nominated[1])):   
                ### NOMINEES ####
                    # NAME
                    path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-field-actor-name > h4::text".format(i+1, 4 +number)
                    nominee_name = ''.join(list_response[years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4
                    list_year.append(1930 + years)         
                    list_category.append(categories[i])
                    list_results.append("Nominee")

                    if categories[i] == "Directing": # For Directing, titles and names are swapped on the website
                        list_films.append(nominee_name)
                    else:
                        list_names.append(nominee_name)
                    
                    # FILM
                    path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-title > span::text".format(i+1, 4 +number)
                    nominee_film = ''.join(list_response[years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                    nominee_film = nominee_film.replace("\n", "") # Cleaning up the string
                    if categories[i] == "Directing":# For Directing, titles and names are swapped on the website
                        list_names.append(nominee_film)
                    else:
                        list_films.append(nominee_film)    
                    
            if len(number_of_film_winner) == 1: # 2 Winners
                ### WINNERS ###       
                # NAME 1st Winner 
                path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2) > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
                nominee_name = ''.join(list_response[years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4
                list_year.append(1930 + years)
                list_category.append(categories[i])
                list_results.append("Tie-winner")

                if categories[i] == "Directing": # For Directing, titles and names are swapped on the website
                    list_films.append(nominee_name)
                else:
                    list_names.append(nominee_name)
                    
                # NAME 2st Winner 
                path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(3) > div.views-field.views-field-field-actor-name > h4::text".format(i+1)
                nominee_name = ''.join(list_response[years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4
                list_year.append(1930 + years)
                list_category.append(categories[i])
                list_results.append("Tie-winner")
                if categories[i] == "Directing": # For Directing, titles and names are swapped on the website
                    list_films.append(nominee_name)
                else:
                    list_names.append(nominee_name)

                
                # FILM
                path_film1 = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(2) > div.views-field.views-field-title > span::text'.format(i+1)
                path_film2 = '#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child(3) > div.views-field.views-field-title > span::text'.format(i+1)

                nominee_film = ''.join(list_response[years].css(path_film1).extract()) # ''.join() transforms the list produced by the selector into a string
                nominee_film = nominee_film.replace("\n", "") # Cleaning up the string
                
                if categories[i] == "Directing":# For Directing, titles and names are swapped on the website
                    list_names.append(nominee_film)
                else:
                    list_films.append(nominee_film)

                winner2_film = ''.join(list_response[years].css(path_film2).extract()) # ''.join() transforms the list produced by the selector into a string
                winner2_film = winner2_film.replace("\n", "") # Cleaning up the string
            
                if categories[i] == "Directing":# For Directing, titles and names are swapped on the website
                    list_names.append(nominee_film)
                else:
                    list_films.append(nominee_film)

                for number in range(int(number_of_film_nominated[1])):   
                    #### NOMINEES ####
                    # NAME
                    path_name = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-field-actor-name > h4::text".format(i+1, 5 +number)
                    nominee_name = ''.join(list_response[years].css(path_name).extract()) # ''.join() transforms the list produced by the selector into a string4
                    list_year.append(1930 + years)
                    list_category.append(categories[i])
                    list_results.append("Nominee")
                    if categories[i] == "Directing": # For Directing, titles and names are swapped on the website
                        list_films.append(nominee_name)
                    else:
                        list_names.append(nominee_name)
                    
                    # FILM
                    path_film = "#quicktabs-tabpage-honorees-0 > div > div.view-content > div:nth-child({}) > div.view-grouping-content > div:nth-child({}) > div.views-field.views-field-title > span::text".format(i+1, 5 +number)
                    nominee_film = ''.join(list_response[years].css(path_film).extract()) # ''.join() transforms the list produced by the selector into a string
                    nominee_film = nominee_film.replace("\n", "") # Cleaning up the string

                    if categories[i] == "Directing":# For Directing, titles and names are swapped on the website
                        list_names.append(nominee_film)
                    else:
                        list_films.append(nominee_film)     

     

print("Done")


Done


['Warner Baxter', 'George Bancroft', 'Chester Morris', 'Paul Muni', 'Lewis Stone', 'Mary Pickford', 'Ruth Chatterton', 'Betty Compson', 'Jeanne Eagels', 'Corinne Griffith', 'Bessie Love', 'Frank Lloyd', 'Lionel Barrymore', 'Harry Beaumont', 'Irving Cummings', 'Frank Lloyd', 'Frank Lloyd', 'Ernst Lubitsch', 'George Arliss', 'George Arliss', 'Wallace Beery', 'Maurice Chevalier', 'Maurice Chevalier', 'Ronald Colman', 'Ronald Colman', 'Lawrence Tibbett', 'Norma Shearer', 'Nancy Carroll', 'Ruth Chatterton', 'Greta Garbo', 'Greta Garbo', 'Norma Shearer', 'Gloria Swanson', 'Lewis Milestone', 'Clarence Brown', 'Clarence Brown', 'Robert Leonard', 'Ernst Lubitsch', 'King Vidor', 'Lionel Barrymore', 'Jackie Cooper', 'Richard Dix', 'Fredric March', 'Adolphe Menjou', 'Marie Dressler', 'Marlene Dietrich', 'Irene Dunne', 'Ann Harding', 'Norma Shearer', 'Norman Taurog', 'Wesley Ruggles', 'Clarence Brown', 'Lewis Milestone', 'Josef Von Sternberg', 'Wallace Beery', 'Fredric March', 'Alfred Lunt', 'Helen

In [199]:
# importing pandas as pd 
import pandas as pd 

  
# dictionary of lists 
dict = {'name': list_names, 'category': list_category, 'film': list_films, 'result': list_results, 'year': list_year} 
    
df = pd.DataFrame(dict)
df = df.set_index(['year','category'])
df 

name                   film   result
year category                                                    
1930 Actor          Warner Baxter         In Old Arizona   Winner
     Actor        George Bancroft            Thunderbolt  Nominee
     Actor         Chester Morris                  Alibi  Nominee
     Actor              Paul Muni            The Valiant  Nominee
     Actor            Lewis Stone            The Patriot  Nominee
...                           ...                    ...      ...
2021 Directing         Chloé Zhao              Nomadland   Winner
     Directing  Thomas Vinterberg          Another Round  Nominee
     Directing      David Fincher                   Mank  Nominee
     Directing    Lee Isaac Chung                 Minari  Nominee
     Directing    Emerald Fennell  Promising Young Woman  Nominee

[1372 rows x 3 columns]

AttributeError: 'str' object has no attribute 'dtype'